In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns; sns.set()

%matplotlib inline

In [2]:
#Read in some CSV files from repository
game = pd.read_csv('data/game.csv')
game = game.set_index('game_id')
team_info = pd.read_csv('data/team_info.csv')
team_info = team_info.set_index('team_id')
#note that abbreviation_x = away, abbreviation_y = home, need to change column names
game = game.merge(team_info[['abbreviation']], left_on='away_team_id', right_on='team_id',right_index=True)
game = game.merge(team_info[['abbreviation']], left_on='home_team_id', right_on='team_id',right_index=True)
game.rename(columns={'abbreviation_x': 'away_team', 'abbreviation_y': 'home_team'}, inplace=True)
game.shape
game.head()
#game_teams_stats = pd.merge(game_teams_stats, game[['type']], on = 'game_id')


,season,type,date_time,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz,away_team,home_team
game_id,,,,,,,,,,,,,,,,
2012030221,20122013,P,2013-05-16,3,6,2,3,home win OT,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,NYR,BOS
2012030222,20122013,P,2013-05-19,3,6,2,5,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,NYR,BOS
2012030225,20122013,P,2013-05-25,3,6,1,3,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,NYR,BOS
2013020380,20132014,R,2013-11-29,3,6,2,3,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,NYR,BOS
2012020179,20122013,R,2013-02-13,3,6,4,3,away win SO,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,NYR,BOS


In [3]:
# create column in game df with binary home / away win; home win = 1, away = 0
game.loc[game['home_goals'] > game['away_goals'], 'side_won'] = int(1)
game.loc[game['home_goals'] < game['away_goals'], 'side_won'] = int(0)

# ensure date_time column to pandas date time and add month column
game['date_time'] = pd.to_datetime(game['date_time'])
game['month'] = game['date_time'].dt.month

game.head()

,season,type,date_time,away_team_id,home_team_id,away_goals,home_goals,outcome,home_rink_side_start,venue,venue_link,venue_time_zone_id,venue_time_zone_offset,venue_time_zone_tz,away_team,home_team,side_won,month
game_id,,,,,,,,,,,,,,,,,,
2012030221,20122013,P,2013-05-16,3,6,2,3,home win OT,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,NYR,BOS,1.0,5
2012030222,20122013,P,2013-05-19,3,6,2,5,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,NYR,BOS,1.0,5
2012030225,20122013,P,2013-05-25,3,6,1,3,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,NYR,BOS,1.0,5
2013020380,20132014,R,2013-11-29,3,6,2,3,home win REG,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,NYR,BOS,1.0,11
2012020179,20122013,R,2013-02-13,3,6,4,3,away win SO,left,TD Garden,/api/v1/venues/null,America/New_York,-4,EDT,NYR,BOS,0.0,2


## Exploratory Analysis of Game Data

In [4]:
# of games in each season - regular season + playoffs; note 2012-2013 is shortened by lockout
games_by_season = game['season'].value_counts()
total_games = sum(games_by_season)
print(games_by_season)

print("Total games in data set:", "{:,}".format(total_games))

20172018    1355
20132014    1323
20152016    1321
20142015    1319
20162017    1317
20122013     806
Name: season, dtype: int64
Total games in data set: 7,441


In [5]:
# calc total home win percentage in data set
home_wins = game['side_won'].sum()
total_wins = game['side_won'].count()
home_win_pct = "{:.2%}".format(home_wins / total_wins)
print("Home win % in 2012-2013 through 2017-2018 seasons:", home_win_pct)

Home win % in 2012-2013 through 2017-2018 seasons: 54.95%


In [6]:
# calc total home win percentage playoffs vs. regular season
tot_reg = game[game.type == 'R'].count()["side_won"]
home_reg = game[(game.type == 'R') & (game.side_won == 1)].sum()["side_won"]
tot_po = game[game.type == 'P'].count()["side_won"]
home_po = game[(game.type == 'P') & (game.side_won == 1)].sum()["side_won"]
home_reg_win_pct = "{:.2%}".format(home_reg / tot_reg)
home_po_win_pct = "{:.2%}".format(home_po / tot_po)
print("Home regular season win pct: " + str(home_reg_win_pct) + ". Home playoff win pct: " \
      + str(home_po_win_pct))

Home regular season win pct: 54.84%. Home playoff win pct: 56.42%


In [7]:
# calc total home win percentage by regular season
home_pct_by_season = {i: game[(game.type == 'R') & (game.side_won == 1) & \
                    (game.season == i)].sum()["side_won"]  \
                    / game[(game.type == 'R') & \
                    (game.season == i)].count()["side_won"] for i in game.season.unique()}
print("\n".join("{}\t{:.2%}".format(k,v) for k,v in home_pct_by_season.items()))

20122013	56.81%
20132014	53.66%
20142015	54.15%
20152016	52.93%
20162017	55.93%
20172018	56.33%


In [8]:
# home win % by month of regular season (need to order with ordered dict)
# tuple is (% home win, number of games played in month)
# how to change formatting of one element in tuple?
reg_season = game[(game.type == 'R')]
home_pct_by_month = {i: (reg_season[(reg_season.side_won == 1) & \
                    (reg_season.month == i)].sum()["side_won"]  \
                    / reg_season[(reg_season.month == i)].count()["month"],  \
                         reg_season[(reg_season.month == i)].count()["month"])
                     for i in reg_season.month.unique()}
print("\n".join("{}\t{}".format(k,v) for k,v in home_pct_by_month.items()))

11	(0.5507389162561577, 1015)
2	(0.5228136882129277, 1052)
1	(0.5409836065573771, 1037)
3	(0.5507669831994156, 1369)
12	(0.5768482490272373, 1028)
4	(0.5598006644518272, 602)
10	(0.5396039603960396, 808)


In [9]:
# home win % by month of playoffs (need to order with ordered dict)
# tuple is (% home win, number of games played in month)

playoffs = game[(game.type == 'P')]
home_pct_by_month = {i: (playoffs[(playoffs.side_won == 1) & \
                    (playoffs.month == i)].sum()["side_won"]  \
                    / playoffs[(playoffs.month == i)].count()["month"],  \
                         playoffs[(playoffs.month == i)].count()["month"])
                     for i in playoffs.month.unique()}
print("\n".join("{}\t{}".format(k,v) for k,v in home_pct_by_month.items()))

5	(0.5850622406639004, 241)
6	(0.6, 40)
4	(0.5381526104417671, 249)


interesting that later rounds of playoffs appear to have higher home win percentages

In [10]:
#goals scored home vs. away
home_goals_pg = reg_season['home_goals'].sum() / reg_season['home_goals'].count()
away_goals_pg = reg_season['away_goals'].sum() / \
            reg_season['home_goals'].count()
differential_pg = home_goals_pg - away_goals_pg
print("Home goals scored: " + str(home_goals_pg) + "\nAway goals scored: " \
      + str(away_goals_pg) + "\nDifferential: " + str(differential_pg))

Home goals scored: 2.9154970337143684
Away goals scored: 2.6431775430473157
Differential: 0.27231949066705274


In [17]:
goals_for_by_team = {i: (reg_season[(reg_season.home_team == i)].sum()['home_goals'] \
                        / reg_season[(reg_season.home_team == i)].count()['home_goals'], \
                        reg_season[(reg_season.away_team == i)].sum()['away_goals'] \
                        / reg_season[(reg_season.away_team == i)].count()['away_goals'],
                        reg_season[(reg_season.home_team == i)].sum()['home_goals'] \
                        / reg_season[(reg_season.home_team == i)].count()['home_goals'] \
                        - reg_season[(reg_season.away_team == i)].sum()['away_goals'] \
                        / reg_season[(reg_season.away_team == i)].count()['away_goals']
                        ) \

                         for i in reg_season.home_team.unique()}

print("\n".join("{}\t{}".format(k,v) for k,v in goals_for_by_team.items()))

BOS	(3.0087336244541483, 2.8820960698689957, 0.12663755458515258)
WSH	(3.1441048034934496, 2.96943231441048, 0.1746724890829694)
PIT	(3.3842794759825328, 2.8777292576419216, 0.5065502183406112)
TBL	(3.2794759825327513, 2.8646288209606987, 0.4148471615720526)
OTT	(2.8777292576419216, 2.6200873362445414, 0.2576419213973802)
WPG	(3.1572052401746724, 2.6331877729257642, 0.5240174672489082)
CAR	(2.5807860262008733, 2.502183406113537, 0.0786026200873362)
CBJ	(2.9519650655021836, 2.7117903930131004, 0.2401746724890832)
CHI	(3.0829694323144103, 2.851528384279476, 0.23144104803493448)
NYI	(3.091703056768559, 2.816593886462882, 0.27510917030567716)
PHI	(2.921397379912664, 2.6157205240174672, 0.30567685589519655)
SJS	(2.9912663755458517, 2.7510917030567685, 0.2401746724890832)
MTL	(2.777292576419214, 2.6419213973799125, 0.1353711790393013)
LAK	(2.777292576419214, 2.567685589519651, 0.20960698689956292)
NJD	(2.493449781659389, 2.331877729257642, 0.161572052401747)
VAN	(2.6026200873362444, 2.445414

In [18]:
# goals per game against at home vs. away by each team in regular season

goals_against_by_team = {i: (reg_season[(reg_season.home_team == i)].sum()['away_goals'] \
                        / reg_season[(reg_season.home_team == i)].count()['away_goals'], \
                         reg_season[(reg_season.away_team == i)].sum()['home_goals'] \
                        / reg_season[(reg_season.away_team == i)].count()['home_goals'],
                        reg_season[(reg_season.home_team == i)].sum()['away_goals'] \
                        / reg_season[(reg_season.home_team == i)].count()['away_goals'] \
                        - reg_season[(reg_season.away_team == i)].sum()['home_goals'] \
                        / reg_season[(reg_season.away_team == i)].count()['home_goals']
                                                       
                            ) \

                         for i in reg_season.home_team.unique()}

print("\n".join("{}\t{}".format(k,v) for k,v in goals_against_by_team.items()))

BOS	(2.388646288209607, 2.646288209606987, -0.2576419213973802)
WSH	(2.3406113537117905, 2.8427947598253276, -0.5021834061135371)
PIT	(2.519650655021834, 2.8209606986899565, -0.3013100436681224)
TBL	(2.6200873362445414, 2.7947598253275108, -0.1746724890829694)
OTT	(2.8777292576419216, 2.9563318777292578, -0.0786026200873362)
WPG	(2.7860262008733625, 2.908296943231441, -0.12227074235807844)
CAR	(2.7467248908296944, 3.078602620087336, -0.3318777292576418)
CBJ	(2.62882096069869, 2.8820960698689957, -0.2532751091703056)
CHI	(2.3580786026200875, 2.834061135371179, -0.47598253275109137)
NYI	(3.039301310043668, 3.03056768558952, 0.00873362445414827)
PHI	(2.611353711790393, 3.096069868995633, -0.4847161572052401)
SJS	(2.4541484716157207, 2.7336244541484715, -0.27947598253275086)
MTL	(2.4017467248908297, 2.921397379912664, -0.519650655021834)
LAK	(2.2445414847161573, 2.558951965065502, -0.3144104803493448)
NJD	(2.519650655021834, 2.9344978165938866, -0.4148471615720526)
VAN	(2.7816593886462884,

## Exploratory Analysis of Game Team Data

In [19]:
game_teams_stats = pd.read_csv('data/game_teams_stats.csv')
game_teams_stats = game_teams_stats.set_index('game_id')
#team_info = pd.read_csv('data/team_info.csv')
#team_info = team_info.set_index('team_id')
game_teams_stats.shape

(14882, 14)

In [20]:
# join with game df to get regular season vs. playoffs
game_teams_stats = pd.merge(game_teams_stats, game[['type']], on = 'game_id')
game_teams_stats = game_teams_stats.join(team_info['abbreviation'], 'team_id')
game_teams_stats.head()

,team_id,HoA,won,settled_in,head_coach,goals,shots,hits,pim,powerPlayOpportunities,powerPlayGoals,faceOffWinPercentage,giveaways,takeaways,type,abbreviation
game_id,,,,,,,,,,,,,,,,
2012030221,3,away,False,OT,John Tortorella,2,35,44,8,3,0,44.8,17,7,P,NYR
2012030221,6,home,True,OT,Claude Julien,3,48,51,6,4,1,55.2,4,5,P,BOS
2012030222,3,away,False,REG,John Tortorella,2,37,33,11,5,0,51.7,1,4,P,NYR
2012030222,6,home,True,REG,Claude Julien,5,32,36,19,1,0,48.3,16,6,P,BOS
2012030223,6,away,True,REG,Claude Julien,2,34,28,6,0,0,61.8,10,7,P,BOS


In [ ]:
#shots home vs away